### 0.) Überblick

Heute lernen wir die Verwendung des Data Fabric kennen. Dafür werden wir:

- Einen Connector initialisieren, der mit einer Datenquelle – konkret einer OPC UA IIoT-Datenquelle – verbunden ist
- Über unseren Client mit dem Connector interagieren, indem wir Daten auslesen und abonnieren
- Mit dem Knowledge Graph interagieren, um die zentral gespeicherten und harmonisierten Metadaten unserer Geräte einzusehen


Dafür benötigen wir aber in einem ersten Schritt eine kurze Einführung in die Komponenten des Data Fabric.

### 1.) Grundlegende Architektur und beteiligte Komponenten

#### **1.1) Grundlegende Architektur des Data Fabric**


Um eine skalierbare, flexible und semantisch integrierte Dateninfrastruktur für IIoT-Anwendungen bereitzustellen, basiert das Data Fabric auf vier zentralen Komponenten:

- **Message Broker:**
Ermöglicht die Kommunikation und dient als zentrales Rückgrat des Datenaustauschs – über die Topic-Typen Request, Response und Telemetry.

- **Connector:**
Ein anpassbares Gateway, das den Datenzugriff auf unterschiedliche IIoT-Geräte und -Protokolle standardisiert und nahtlos in das Data Fabric integriert.

- **Knowledge Graph:**
Dient als semantischer Datenkatalog, der IIoT-Geräte, Services und Datenflüsse kontextualisiert und alle Komponenten im System verknüpft.

- **Zentraler Client/Agent:**
Steuert Connectoren und fragt den Knowledge Graph ab, um Aktionen auszulösen und Kontextinformationen abzurufen.

<p align="center">
  <img src="../docs/img/architecture.png" width="400"><br>
  <em>Figure: Überblick über die Komponenten des Data Fabric und deren Zusammenspiel</em>
</p>

#### **1.2) Überblick zu den Komponenten**

Um zunächst sicherzustellen, dass unser Data Fabric noch nicht aktiv ist, prüfen wir den aktuellen Zustand des zentralen Knowledge Graphs: [Knowledge Graph](http://10.157.82.21:7200/sparql?name=full_graph&infer=true&sameAs=true&query=PREFIX%20df%3A%20%3Chttp%3A%2F%2Fstephantrattnig.org%2Fdata_fabric_ontology%23%3E%0APREFIX%20df_instance%3A%20%3Chttp%3A%2F%2Fstephantrattnig.org%2Finstances%23%3E%0APREFIX%20rdfs%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0APREFIX%20rdf%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23%3E%0A%0ACONSTRUCT%20%7B%0A%20%20%3Fsubject%20%3Fpredicate%20%3Fobject%20.%0A%20%20%3Fsubject%20rdfs%3Alabel%20%3Flabel%20.%0A%7D%0AWHERE%20%7B%0A%20%20%3Fsubject%20%3Fpredicate%20%3Fobject%20.%0A%20%20FILTER%20(%3Fpredicate%20!%3D%20rdf%3Atype)%20%20%23%20Filter%20out%20rdf%3Atype%20triples%0A%20%20%20%20FILTER%20NOT%20EXISTS%20%7B%0A%20%20%3Fsubject%20%3Fpredicate%20%3Fobject%20.%0A%20%20FILTER(%3Fpredicate%20IN%20(df%3AhasTopicType%2C%20df%3AhasRole%2C%20df%3AhasCapability%2C%20df%3AallowsCapability%2C%20df%3AhasAccessMode%2C%20df%3AprovidesDataPoint))%0A%7D%0A%20%20FILTER%20(%0A%20%20STRSTARTS(STR(%3Fsubject)%2C%20%22http%3A%2F%2Fstephantrattnig.org%2Finstances%23%22)%20%7C%7C%0A%20%20%3Fsubject%20%3D%20df%3AMessageBrokerInstance%0A%09)%0A%20%20OPTIONAL%20%7B%20%3Fsubject%20rdfs%3Alabel%20%3Flabel%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fobject%20rdfs%3Alabel%20%3FlabelObject%20%7D%0A%7D)

Außerdem sehen wir, dass derzeit keine Telemetriedaten übertragen werden, z. B. über das Topic: [telemetry.ModuleOPCUA.opcua_testserver](http://10.157.101.85:8080/ui/clusters/local/all-topics/telemetry.ModuleOPCUA.opcua_testserver/messages?filterQueryType=STRING_CONTAINS&attempt=2&limit=100&page=0&seekDirection=TAILING&keySerde=String&valueSerde=String&seekType=LATEST)

Ein typisches Tool zur Inspektion von OPC UA Servern und zur Überprüfung des laufenden Betriebs ist UA Expert (leider kein Link verfügbar).

Relevante Komponenten:
- [Kafka_Broker](http://10.157.101.85:8080/)
- [GraphDB](http://10.157.82.21:7200/)
- [Minio](http://10.157.101.85:9001/browser)


#### **1.3) Data Fabric Ontologie**

oder: *Konstruktionsplan des zentralen Knowledge Graphs*

Die Ontologie bildet das semantische Fundament unseres zentralen Knowledge Graphs. Sie definiert alle relevanten Klassen (z. B. Device, Connector, DataPoint) sowie deren Beziehungen (z. B. hasDataPoint, connectedTo). Dadurch wird sichergestellt, dass alle Komponenten und Datenflüsse im Data Fabric einheitlich beschrieben und vernetzt werden können. Hier können wir uns einen Überblick über die Ontologie verschaffen: [Data Fabric Ontology](http://10.157.82.21:7200/graphs-visualizations?saved=819bea8e8d93440b814181d624d0f7e7)

### 2.) Initialisierung des Connectors und Fabric-Mechanismus


#### **2.1) Initialisierung**

Um eine neue IIoT-Datenquelle in das Data Fabric einzubinden, muss ein Connector initialisiert werden. Dieser verbindet sich mit der Datenquelle und *registriert* diese im Datenkatalog. 

💡 Hinweis: Der Start des Connectors erfolgt separat – verwende dafür z. B. das Notebook connector_start.ipynb oder die Command Line, um die Logs aufzuzeigen. 

Überprüfe den zentralen Knowledge Graph: [Knowledge Graph](http://10.157.82.21:7200/sparql?name=full_graph&infer=true&sameAs=true&query=PREFIX%20df%3A%20%3Chttp%3A%2F%2Fstephantrattnig.org%2Fdata_fabric_ontology%23%3E%0APREFIX%20df_instance%3A%20%3Chttp%3A%2F%2Fstephantrattnig.org%2Finstances%23%3E%0APREFIX%20rdfs%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0APREFIX%20rdf%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23%3E%0A%0ACONSTRUCT%20%7B%0A%20%20%3Fsubject%20%3Fpredicate%20%3Fobject%20.%0A%20%20%3Fsubject%20rdfs%3Alabel%20%3Flabel%20.%0A%7D%0AWHERE%20%7B%0A%20%20%3Fsubject%20%3Fpredicate%20%3Fobject%20.%0A%20%20FILTER%20(%3Fpredicate%20!%3D%20rdf%3Atype)%20%20%23%20Filter%20out%20rdf%3Atype%20triples%0A%20%20%20%20FILTER%20NOT%20EXISTS%20%7B%0A%20%20%3Fsubject%20%3Fpredicate%20%3Fobject%20.%0A%20%20FILTER(%3Fpredicate%20IN%20(df%3AhasTopicType%2C%20df%3AhasRole%2C%20df%3AhasCapability%2C%20df%3AallowsCapability%2C%20df%3AhasAccessMode%2C%20df%3AprovidesDataPoint))%0A%7D%0A%20%20FILTER%20(%0A%20%20STRSTARTS(STR(%3Fsubject)%2C%20%22http%3A%2F%2Fstephantrattnig.org%2Finstances%23%22)%20%7C%7C%0A%20%20%3Fsubject%20%3D%20df%3AMessageBrokerInstance%0A%09)%0A%20%20OPTIONAL%20%7B%20%3Fsubject%20rdfs%3Alabel%20%3Flabel%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fobject%20rdfs%3Alabel%20%3FlabelObject%20%7D%0A%7D)

#### **2.2) Fabric-Mechanismus**

**Was ist passiert?**

Das Fabric-Mechanismus hat gegriffen. Wie in der untenstehenden Abbildung zu sehen ist, sorgen drei unterschiedliche Mechanismen dafür, dass der zentrale Knowledge Graph stets aktuell bleibt – auch im Hinblick auf verteilte IIoT-Datenquellen wie unseren OPC UA-Server.

- **metadata_init():** Diese Subroutine sorgt dafür, dass der Connector initial in den Knowledge Graph eingebunden wird. Dabei werden vor allem Metadaten übertragen, die den Connector charaketerisieren - weniger die darunterliegende Datenquelle.
- **metadata_crawl():** Diese Subroutine ist für jeden Typ an möglicher IIoT-Datenquelle speziell angepasst. Sie sorgt dafür, dass die Datenquelle nach möglichen Datenpunkten durchsucht wird und registriert diese zentral im Wissensgraphen. Dabei ist folgendes hervorhebenswert: (1) es findet bereits eine *Angleichung* der Datenpunkte statt und (2) der Prozess findet vollautomatisiert statt.

... die Subroutine **monitor_subscription()** lernen wir später kennen.

<p align="center">
  <img src="../docs/img/Journal1-metadata_contributions.drawio.png" width="500"><br>
  <em>Figure: Detailansicht des Fabric-Mechanismus</em>
</p>



### 3.) Interaktion mit dem Konnektor

#### **3.1) Grundlagen**

Wie kontrollieren wir den Connector auf eine skalierbare Weise von einem zentralen Punkt aus?
→ Antwort: Über strukturierte Nachrichten

Eine Nachricht ist ein strukturiertes Datenpaket, das eine bestimmte Aktion beschreibt – z. B. einen Leseauftrag an eine OPC UA-Datenquelle.
Es enthält Metadaten wie:

    die Nachrichtenversion

    einen eindeutigen Identifikator

    sowie einen Payload, z. B. eine Liste von Datenpunkten (DataPoints), die gelesen werden sollen.

Die Kommunikation erfolgt über ein zentrales Nachrichtenschema, das sicherstellt, dass alle Nachrichten einheitlich verarbeitet werden.

<p align="center">
  <img src="../docs/img/Journal1-Messaging_MsgModelHierarchy.drawio.png" width="800"><br>
  <em>Abbildung: Nachrichtenschema</em>
</p>

In [1]:
# ein paar nervige Import-statements. Python...
import os
import sys

#import matplotlib.pyplot as plt
import uuid
from os import environ

# Adjust depending on your folder structure — go up to project root
sys.path.append(os.path.abspath(".."))

#from messaging.base_payload_examples import example_read_payload
#from messaging.base_payload_generator import generate_read_payload
#from messaging.datamodel import OPCUAUnsubscribeCommandPayload, OPCUAReadCommandPayload

from connector.messages.datamodel_utils import (
        SubscriptionRegisterRequest,
        SubscriptionUnregisterRequest,
)
from connector_client import ConnectorClient

# initializing the client
client = ConnectorClient(bootstrap_servers=[environ.get("KAFKA_BROKER")])

#### **3.2) Daten auslesen (ReadCommand)**

Wir definieren eine Nachricht und senden diese an unsere IIoT-Datenquelle (OPC UA Server)

In [1]:
# definieren einer Nachricht für einen OPC-UA-Server
read_base_payload = OPCUAReadCommandPayload(
        device_origin="client",
        nodeid_list = ["ns=2;i=59", "ns=2;i=61"]
    )

NameError: name 'OPCUAReadCommandPayload' is not defined

In [ ]:
# senden der Nachricht und darstellen der Rückmeldung
output = await client.read_data_source(read_base_payload)

**Was ist passiert?**

--> Eine Nachricht wurde gesendet (see Beispiel) und eine darauffolgende Antwort wurde empfangen.

```json
{
	"version": "1.0.0",
	"payload": {
		"type": "ReadCommand",
		"base_payload": {
			"type": "OPCUAReadCommandPayload",
			"device_origin": "client",
			"nodeid_list": [
				"ns=2;i=59",
				"ns=2;i=61"
			]
		}
	},
	"type": "RequestMessage",
	"message_id": "d0800e53-d3c3-485c-bbfb-7c4a4f81ad61",
	"correlation_id": "31ebdb1f-2d45-4f18-b4ad-dac4b69b5ea9",
	"timestamp": "2025-04-08T20:46:33.114375"
}
```

Hinweis: Überprüfe die Topics [request.ModuleOPCUA.opcua_testserver](http://10.157.101.85:8080/ui/clusters/local/all-topics/request.ModuleOPCUA.opcua_testserver/messages?filterQueryType=STRING_CONTAINS&attempt=2&limit=100&page=0&seekDirection=FORWARD&seekType=BEGINNING) und [response.ModuleOPCUA.opcua_testserver](http://10.157.101.85:8080/ui/clusters/local/all-topics/response.ModuleOPCUA.opcua_testserver/messages?keySerde=String&valueSerde=String&limit=100)

#### **3.3) Daten Abonnieren (SubscribeCommand)**

Ein typischer Weg, um eine robuste Datenpipeline aufzubauen, sind Subscriptions (Abonnements). Eine Subscription ist eine Art „Vertrag“ mit einer Datenquelle, bei dem Daten fortlaufend gesendet werden. Sobald neue Daten verfügbar sind, werden sie automatisch übermittelt.

Erneut müssen wir zuerst unsere Nachricht definieren, bevor sie versendet wird.

In [7]:
# definieren einer Subscribe-Nachricht für einen OPC-UA-Server
subscription_id = uuid.uuid4()
print(f"Subscription Identifier: {subscription_id}")
example_subscribe_payload = SubscriptionRegisterRequest(
    datapoint_identifier=["ns=2;i=62"],
    device_origin="client",
    subscription_identifier=subscription_id
)

Subscription Identifier: c1481986-62c1-482e-9b38-f60822513913


In [8]:
# senden der Nachricht und darstellen der Rückmeldung
output = await client.subscribe_data_source(base_payload=example_subscribe_payload)

Correlation_id of sent message: c644fd90-4204-432b-b281-5cf832bf8432
waiting for a message
Received message
{'version': '1.0.0', 'payload': {'type': 'CommandResponse', 'base_payload': {'type': 'OPCUASubscribeCommandResponse', 'device_origin': 'connector2', 'subscription_id': 'c1481986-62c1-482e-9b38-f60822513913', 'subscription_results': {'ns=2;i=62': 'success'}}}, 'type': 'ResponseMessage', 'message_id': 'a30d2034-2701-4931-a8b5-78f8f1abb36f', 'correlation_id': 'c644fd90-4204-432b-b281-5cf832bf8432', 'timestamp': '2025-04-27T15:29:02.741244', 'status_code': 'Valid', 'service_id': 'opcua_testserver'}


ValidationError: 3 validation errors for MsgModel
TelemetryMessage
  Value error, Unknown payload type: OPCUASubscribeCommandResponse [type=value_error, input_value={'version': '1.0.0', 'pay...id': 'opcua_testserver'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error
ResponseMessage
  Value error, Unknown payload type: OPCUASubscribeCommandResponse [type=value_error, input_value={'version': '1.0.0', 'pay...id': 'opcua_testserver'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error
RequestMessage
  Value error, Unknown payload type: OPCUASubscribeCommandResponse [type=value_error, input_value={'version': '1.0.0', 'pay...id': 'opcua_testserver'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

**Was ist passiert?**

Durch das Senden eines Subscribe-Befehls an den Connector wurde ein kontinuierlicher Datenstrom eingerichtet. Überprüfe das Telemetry-Topic, um festzustellen, ob bereits Daten übertragen werden: [telemetry.ModuleOPCUA.opcua_testserver](http://10.157.101.85:8080/ui/clusters/local/all-topics/telemetry.ModuleOPCUA.opcua_testserver/messages?filterQueryType=STRING_CONTAINS&attempt=2&limit=100&page=0&seekDirection=TAILING&keySerde=String&valueSerde=String&seekType=LATEST)

Da Subscriptions keine einmaligen Vorgänge sind, sondern über einen längeren Zeitraum bestehen bleiben können, werden sie im zentralen Knowledge Graph nachverfolgt. Überprüfe, ob die Subscription registriert wurde: [Subscription View](http://10.157.82.21:7200/sparql?name=full_graph&infer=true&sameAs=true&query=PREFIX%20df%3A%20%3Chttp%3A%2F%2Fstephantrattnig.org%2Fdata_fabric_ontology%23%3E%0APREFIX%20df_instance%3A%20%3Chttp%3A%2F%2Fstephantrattnig.org%2Finstances%23%3E%0APREFIX%20rdfs%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0APREFIX%20rdf%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23%3E%0A%0ACONSTRUCT%20%7B%0A%20%20%3Fsubject%20%3Fpredicate%20%3Fobject%20.%0A%20%20%3Fsubject%20rdfs%3Alabel%20%3Flabel%20.%0A%7D%0AWHERE%20%7B%0A%20%20%3Fsubject%20%3Fpredicate%20%3Fobject%20.%0A%20%20FILTER%20(%3Fpredicate%20!%3D%20rdf%3Atype)%20%20%23%20Filter%20out%20rdf%3Atype%20triples%0A%20%20%20%20FILTER%20NOT%20EXISTS%20%7B%0A%20%20%3Fsubject%20%3Fpredicate%20%3Fobject%20.%0A%20%20FILTER(%3Fpredicate%20IN%20(df%3AhasTopicType%2C%20df%3AhasRole%2C%20df%3AhasCapability%2C%20df%3AallowsCapability%2C%20df%3AhasAccessMode%2C%20df%3AprovidesDataPoint))%0A%7D%0A%20%20FILTER%20(%0A%20%20STRSTARTS(STR(%3Fsubject)%2C%20%22http%3A%2F%2Fstephantrattnig.org%2Finstances%23%22)%20%7C%7C%0A%20%20%3Fsubject%20%3D%20df%3AMessageBrokerInstance%0A%09)%0A%20%20OPTIONAL%20%7B%20%3Fsubject%20rdfs%3Alabel%20%3Flabel%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fobject%20rdfs%3Alabel%20%3FlabelObject%20%7D%0A%7D)

#### **3.4) Datenabonnements aufheben (Unsubscribe)**

Manchmal ändern sich die Anforderungen an die Datenerfassung häufig – zum Beispiel, weil sich der Fokus auf bestimmte Variablen im Laufe datengetriebener Anwendungsfälle verschiebt. In anderen Fällen müssen Daten nur für einen bestimmten Zeitraum erfasst werden.

Wir werden nun unsere oben erstellte Subscription schließen (also quasi unser Abonnement *kündigen*).

In [ ]:
# die vorhin zurückgegebene Subscription ID
print(output.root.payload.base_payload.subscription_id)

In [4]:
# definieren einer Unsubscribe-Nachricht für einen OPC-UA-Server
example_unsubscribe_payload = SubscriptionUnregisterRequest(
    subscription_identifier = output.root.payload.base_payload.subscription_id,
    device_origin="client"
)

NameError: name 'output' is not defined

In [6]:
# senden der Nachricht und darstellen der Rückmeldung
output = await client.unsubscribe_data_source(base_payload = example_unsubscribe_payload)

NameError: name 'example_unsubscribe_payload' is not defined

**Was ist passiert?**

Durch das Senden eines Unsubscribe-Befehls an den Connector wurde die Subscription geschlossen. Überprüfe erneut das Telemetrie-Topic, ob weiterhin Daten übertragen werden: [telemetry.ModuleOPCUA.opcua_testserver](http://10.157.101.85:8080/ui/clusters/local/all-topics/telemetry.ModuleOPCUA.opcua_testserver/messages?filterQueryType=STRING_CONTAINS&attempt=2&limit=100&page=0&seekDirection=TAILING&keySerde=String&valueSerde=String&seekType=LATEST)

Überprüfe außerdem im Knowledge Graph, ob die Subscription noch sichtbar ist: [Subscription View](http://10.157.82.21:7200/sparql?name=full_graph&infer=true&sameAs=true&query=PREFIX%20df%3A%20%3Chttp%3A%2F%2Fstephantrattnig.org%2Fdata_fabric_ontology%23%3E%0APREFIX%20df_instance%3A%20%3Chttp%3A%2F%2Fstephantrattnig.org%2Finstances%23%3E%0APREFIX%20rdfs%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0APREFIX%20rdf%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23%3E%0A%0ACONSTRUCT%20%7B%0A%20%20%3Fsubject%20%3Fpredicate%20%3Fobject%20.%0A%20%20%3Fsubject%20rdfs%3Alabel%20%3Flabel%20.%0A%7D%0AWHERE%20%7B%0A%20%20%3Fsubject%20%3Fpredicate%20%3Fobject%20.%0A%20%20FILTER%20(%3Fpredicate%20!%3D%20rdf%3Atype)%20%20%23%20Filter%20out%20rdf%3Atype%20triples%0A%20%20%20%20FILTER%20NOT%20EXISTS%20%7B%0A%20%20%3Fsubject%20%3Fpredicate%20%3Fobject%20.%0A%20%20FILTER(%3Fpredicate%20IN%20(df%3AhasTopicType%2C%20df%3AhasRole%2C%20df%3AhasCapability%2C%20df%3AallowsCapability%2C%20df%3AhasAccessMode%2C%20df%3AprovidesDataPoint))%0A%7D%0A%20%20FILTER%20(%0A%20%20STRSTARTS(STR(%3Fsubject)%2C%20%22http%3A%2F%2Fstephantrattnig.org%2Finstances%23%22)%20%7C%7C%0A%20%20%3Fsubject%20%3D%20df%3AMessageBrokerInstance%0A%09)%0A%20%20OPTIONAL%20%7B%20%3Fsubject%20rdfs%3Alabel%20%3Flabel%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fobject%20rdfs%3Alabel%20%3FlabelObject%20%7D%0A%7D)

#### **3.5) Fortgeschritteneres Datenmanagement**

Neben der reinen *Steuerung* unserer Datenquellen können auch komplexere Interaktionen mit unseren IIoT-Geräten durchgeführt werden. Ein Beispiel: Wir abonnieren eine Datenquelle über einen *Request*-Befehl, lauschen dem Datenstrom über den *Telemetry*-Kanal 
und geben schließlich die erfassten Daten zurück.

In [ ]:
# subscribe to a data source and collect the data for processing
output = await client.collect_data_for_duration(example_subscribe_payload, duration=3)

**Was ist passiert?**

Während wir den obigen Client-Befehl ausgeführt haben, wurde eine Subscription zur Datenquelle aufgebaut, Daten wurden gesammelt und nach einer definierten Zeitspanne wieder gestoppt. Daher erwarten wir in unserer Ausgabe mehrere Datenpunkte, mit denen wir nun weiterarbeiten können.

Ein Beispiel: Wir können die Daten visualisieren – z. B. in einem Diagramm.

In [ ]:
# Plotting
pivot_output = output.pivot(index="timestamp", columns="nodeid", values="value")
pivot_output.plot(figsize=(10, 4))
plt.tight_layout()
plt.show()

### 4.) Interaktion mit dem Knowledge Graph

#### **4.1) Grundlagen**

**Was passiert, wenn mehrere IIoT-Geräte angebunden werden sollen?**

In nahezu jedem industriell relevanten Anwendungsszenario wird das Datenmanagement nicht für nur eine, sondern für mehrere verteilte IIoT-Datenquellen gleichzeitig durchgeführt. In diesem Fall initialisieren wir mehrere Connectoren, wobei jeder Connector eine spezifische Datenquelle repräsentiert. So entsteht ein skalierbares und flexibles Netzwerk aus Datenquellen, das zentral verwaltet werden kann.

💡 Hinweis: Der Start der weiteren Connectoren erfolgt separat – verwende dafür z. B. das Notebook connector_start.ipynb oder die Command Line, um die Logs aufzuzeigen. 

In [2]:
# wir benötigen wieder ein paar python imports...
import os
import sys
import uuid
from os import environ

sys.path.append(os.path.abspath(".."))

#from messaging.datamodel import OPCUAReadCommandPayload
from connector.messages.datamodel_utils import (
        SubscriptionRegisterRequest,
        SubscriptionUnregisterRequest,
)
from connector_client import ConnectorClient
from sparql_queries import *

client = ConnectorClient(bootstrap_servers=[environ.get("KAFKA_BROKER")])
await client.load_connector_config()
await client.return_connectors()

dict_keys(['injection_molding', 'cnc_milling', 'assembly_station', 'idscamera_trainee_buero', 'opcua_testserver'])

#### **4.2) Abfragen am Wissensgraphen**

Unsere IIoT-Datenquellen sind im Wissensgraphen registriert. Parallel zu der grafischen Darstellung, lassen sich diese auch per Befehl gezielt *auslesen* (engl. *query*).

In [18]:
# abfragen aller im Knowledge Graph registrierten Datenquellen
await client.query_graphdb(build_all_devices_query())

,device,label,deviceAddress,communicationProtocol,deviceModelName,deviceManufacturer
0,2c3ec1cc-981a-4886-a3b4-bb86ff628902,Device: IMM-5000,10.157.101.85:4843,opc.tcp,PlastiTech AG,imm5000
1,f79af31d-3b1f-41a6-b006-d2f7476f8a45,Device: CNC-3000,10.157.101.85:4842,opc.tcp,MyCompany Precision Tools,cnc3000
2,1c1f304d-26e2-4764-b615-b277968d6c72,Device: AS-100,10.157.101.85:4841,opc.tcp,AssemCo GmbH,as100
3,5ad632cd-ee4f-4292-a441-69532d20b174,Device: IDS Camera U3 C30Cx,10.157.82.10,Basler Python Library with GENICAM,U3_30Cx_C,Basler
4,f75b33b5-ddbc-4e09-9e90-6c683aebb58e,Device: CX2030,10.157.101.85:4840,opc.tcp,Beckhoff Automation GmbH,cx2030
5,2a84695f-34ea-4b53-a9b7-8f955343fbf3,Device: CX2030,10.157.101.85:4840,opc.tcp,Beckhoff Automation GmbH,cx2030
6,b21ab301-7ac8-413e-b6fb-c953e266ee29,Device: CX2030,10.157.101.85:4840,opc.tcp,Beckhoff Automation GmbH,cx2030
7,73bc0c13-8812-42d3-af3c-25a422c43582,Device: CX2030,10.157.101.85:4840,opc.tcp,Beckhoff Automation GmbH,cx2030
8,acf73a3d-5d5b-479b-8acc-a2679f580976,Device: CX2030,10.157.101.85:4840,opc.tcp,Beckhoff Automation GmbH,cx2030
9,a7e57515-8616-46a3-8188-91cf07c998ed,Device: CX2030,10.157.101.85:4840,opc.tcp,Beckhoff Automation GmbH,cx2030


In [8]:
# abfragen aller im Wissensgraph registrierten Datenpunkte
await client.query_graphdb(build_connector_datapoint_query())

,connectorId,dataPointName,dataPointIdentifier
0,cnc_milling,Axis_X_Position,ns=2;i=58
1,cnc_milling,Axis_Y_Position,ns=2;i=60
2,cnc_milling,Axis_Z_Position,ns=2;i=62
3,cnc_milling,CoolantFlowRate,ns=2;i=64
4,cnc_milling,FeedRate,ns=2;i=54
5,cnc_milling,SpindleSpeed,ns=2;i=52
6,cnc_milling,ToolTemperature,ns=2;i=56
7,cnc_milling,ToolWear,ns=2;i=66


#### **4.3) Interaktion mit mehreren IIoT-Datenquellen**

Erneut können wir, wie bereits zuvor, einer Datenquelle eine Nachricht zusenden. Dabei sind wir nun aber flexibel hinsichtlich der Datenquelle - können diese also gezielt *auswählen*.

In [8]:
# Auswahl der IIoT-Datenquelle
await client.switch_topic_config("cnc_milling")

In [9]:
# Definieren einer Nachricht an diese Datenquelle
read_base_payload = OPCUAReadCommandPayload(
        device_origin="client",
        nodeid_list = ["ns=2;i=56", "ns=2;i=66"]
    )

output = await client.read_data_source(read_base_payload)

Matching response received:
{
    "type": "OPCUAReadCommandResponse",
    "device_origin": "connector",
    "value_dict": {
        "ns=2;i=56": 21.978097478523694,
        "ns=2;i=66": 1.0
    }
}


#### **4.x) Eine Datenquelle abonnieren**

In [10]:
subscription_id = uuid.uuid4()
# Eine Nachricht definieren
subscription_id = uuid.uuid4()
print(f"Subscription Identifier: {subscription_id}")
example_subscribe_payload = SubscriptionRegisterRequest(
    datapoint_identifier=["ns=2;i=62"],
    device_origin="client",
    subscription_identifier=subscription_id
)

await client.switch_topic_config("cnc_milling")
output = await client.subscribe_data_source(example_subscribe_payload)

Subscription Identifier: 6b4645ea-0ab0-41a4-b5bf-087c5a2e3e47
Correlation_id of sent message: c21e4c90-c0d4-4278-9623-10d4dfa6d4e7
waiting for a message
Received message
{'version': '1.0.0', 'payload': {'type': 'CommandResponse', 'base_payload': {'type': 'OPCUASubscribeCommandResponse', 'device_origin': 'connector2', 'subscription_id': '6b4645ea-0ab0-41a4-b5bf-087c5a2e3e47', 'subscription_results': {'ns=2;i=62': 'success'}}}, 'type': 'ResponseMessage', 'message_id': '41369da8-4f30-40cd-996d-287eb06311d1', 'correlation_id': 'c21e4c90-c0d4-4278-9623-10d4dfa6d4e7', 'timestamp': '2025-04-15T13:14:22.998360', 'status_code': 'Valid', 'service_id': 'cnc_milling'}
Matching response received:
{
    "type": "OPCUASubscribeCommandResponse",
    "device_origin": "connector2",
    "subscription_id": "6b4645ea-0ab0-41a4-b5bf-087c5a2e3e47",
    "subscription_results": {
        "ns=2;i=62": "success"
    }
}


#### **4.4) Mehrere Datenquellen abonnieren (experimentell)**

In [4]:
# Eine Nachricht definieren
subscription_id = uuid.uuid4()
print(f"Subscription Identifier: {subscription_id}")
example_subscribe_payload = SubscriptionRegisterRequest(
    datapoint_identifier=["ns=2;i=62"],
    device_origin="client",
    subscription_identifier=subscription_id
)

# Subscription zu der CNC milling machine
await client.switch_topic_config("cnc_milling")
output = await client.subscribe_data_source(example_subscribe_payload)

# Subscription zu der injection molding machine
await client.switch_topic_config("injection_molding")
output = await client.subscribe_data_source(example_subscribe_payload)


example_subscribe_payload = SubscriptionRegisterRequest(
    datapoint_identifier= ["ns=2;i=53", "ns=2;i=55"],
    device_origin="client",
    subscription_identifier=subscription_id
)

# Subscription zu der assembly station
await client.switch_topic_config("assembly_station")
output = await client.subscribe_data_source(example_subscribe_payload)

Subscription Identifier: bb313b21-b4c9-495b-8c52-b2adb3ea9be2
Correlation_id of sent message: 28be6a6c-b017-4aad-af06-9d4e437818df
waiting for a message
Received message
{'version': '1.0.0', 'payload': {'type': 'CommandResponse', 'base_payload': {'type': 'OPCUASubscribeCommandResponse', 'device_origin': 'connector2', 'subscription_id': 'bb313b21-b4c9-495b-8c52-b2adb3ea9be2', 'subscription_results': {'ns=2;i=62': 'success'}}}, 'type': 'ResponseMessage', 'message_id': '33bcecf6-d10a-4d4c-95aa-2b04f8f019a0', 'correlation_id': '28be6a6c-b017-4aad-af06-9d4e437818df', 'timestamp': '2025-04-13T21:39:06.291239', 'status_code': 'Valid', 'service_id': 'cnc_milling'}
Matching response received:
{
    "type": "OPCUASubscribeCommandResponse",
    "device_origin": "connector2",
    "subscription_id": "bb313b21-b4c9-495b-8c52-b2adb3ea9be2",
    "subscription_results": {
        "ns=2;i=62": "success"
    }
}
Correlation_id of sent message: f11c8ead-1f4b-4d7c-9aef-56ce4d9ea8e9
waiting for a message
Re

In [5]:
example_subscribe_payload = SubscriptionRegisterRequest(
    datapoint_identifier=["new_cam"],
    device_origin="client",
    subscription_identifier=subscription_id
)


await client.switch_topic_config("idscamera_trainee_buero")
output = await client.subscribe_data_source(example_subscribe_payload)

Correlation_id of sent message: 35ae053d-04d0-49f7-9814-25f1a7d2560e
waiting for a message
Received message
{'version': '1.0.0', 'payload': {'type': 'CommandResponse', 'base_payload': {'type': 'SubscriptionRegisterResponse', 'device_origin': 'connector', 'response': 'success'}}, 'type': 'ResponseMessage', 'message_id': '0d0a5479-443f-457c-af5c-730ec0d2de38', 'correlation_id': '35ae053d-04d0-49f7-9814-25f1a7d2560e', 'timestamp': '2025-04-13T21:39:09.796114', 'status_code': 'Valid', 'service_id': 'idscamera_trainee_buero'}
Matching response received:
{
    "type": "SubscriptionRegisterResponse",
    "device_origin": "connector",
    "response": "success"
}


### 5.) Zukünftige Entwicklung

Worin liegen die Stärken dieser Architektur?

Sie ermöglicht die **Interoperable Verwendung von IIoT-Datenquellen**! Interoperabel lässt sich in folgende *Fähigkeiten* gliedern:

1) ✅ technische Interoperabilität (verschiedene Schnittstellen)
2) ✅ syntaktische Interoperabilität (gleichartige Kommunikation - alle Datenquellen sprechen dieselbe Sprache)
3) ✅ semantische Interoperabilität (gleiches Kontextverständnis aller Datenquellen)
4) ✅ organisatorische Interoperabilität (flexibel Einsetzbar, skalierbar und gleichzeitig Nutzbar)

Zukünftige Erweiterbarkeit des Ansatzes:

- Die Herangehensweise über vorentwickelte Connectoren ermöglicht die Skalierung auf weitere Datenquellen (ggfs. kurz die Kamera vorstellen?)
- Der Ausbau des Clients ermöglicht eine Nutzerfreundliche Interaktion mit einem komplett heterogenen
- Erweiterbarkeit um weitere Ontologien, welche direkt aufbauend auf den Datenquellen weitere Semantik erzeugen
- Anstelle eines Clients kann ein LLM-basierter Agent die Kontrolle über das Data Fabric "übernehmen".